In [0]:
import os
import keras
from keras import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib as mpl
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import UpSampling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from skimage.io import imsave, imread
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy import ndimage
from scipy.misc import imresize
from keras.layers.convolutional import Deconv2D as Conv2DTranspose
from keras.models import Model
from keras.layers import Add
from keras.callbacks import ModelCheckpoint,TensorBoard
from keras.optimizers import Adam
from keras.models import Sequential

np.random.seed(1337)

In [0]:
resize_h=256
resize_w=256

def takingInput():
    total=len(os.listdir('melanoma/'))+len(os.listdir('others/'))
    print(total)
    imgs=[]
    imgs_mask=[]
    i = 0
    images = os.listdir('melanoma/')
    for image_name in images:
        image_mask_name = image_name.split('.')[0]
        image_mask_name = image_mask_name+"_segmentation.png"
        img = ndimage.imread('melanoma/'+image_name,mode = 'L')
        img = cv2.resize(img, (resize_h, resize_w))
        img_mask = ndimage.imread('gt/'+image_mask_name,mode = 'L')
        img_mask = imresize(img_mask,(resize_h,resize_w))
        ret2,th2 = cv2.threshold(img_mask,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        img = img_to_array(img)
        img_mask=img_to_array(th2)
        imgs.append(img)
        imgs_mask.append(img_mask)
        print('Done: {0}/{1} images'.format(i, total))
        i += 1
    images = os.listdir('others/')
    for image_name in images:
        image_mask_name = image_name.split('.')[0] 
        image_mask_name = image_mask_name+"_segmentation.png"
        img = ndimage.imread('others/'+image_name,mode = 'L')
        img = cv2.resize(img, (resize_h, resize_w))
        img_mask = ndimage.imread('gt/'+image_mask_name,mode = 'L')
        img_mask=cv2.resize(img_mask,(resize_h,resize_w))
        ret2,th2 = cv2.threshold(img_mask,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        img = img_to_array(img)
        
        img_mask=img_to_array(th2)
        imgs.append(img)
        imgs_mask.append(img_mask)

        print('Done: {0}/{1} images'.format(i, total))
        i += 1
    imgs = np.array(imgs, dtype="float")
    imgs_mask = np.array(imgs_mask,dtype="float")
    return imgs, imgs_mask

X1, Y = takingInput()

In [0]:
np.save( 'imgs_train.npy', X1)
np.save('imgs_mask_train.npy', Y)

In [0]:
X1 = np.load('../imgs_train.npy')
Y = np.load('../imgs_mask_train.npy')

In [0]:
Y1 = Y/255.

In [0]:
Y.shape

(2000, 256, 256, 1)

In [0]:
X = X1/255.

In [0]:
Y.shape

(2000, 256, 256, 1)

In [0]:
(trainX, testX, trainY, testY) = train_test_split(X,Y1, test_size=0.20, random_state=42)

In [0]:
np.unique(trainY)

array([ 0.,  1.])

In [0]:
smooth = 1.
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -K.log(dice_coef(y_true, y_pred))

In [0]:
def get_unet():
    inputs = Input((None, None, 1))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    
    up6 = Add()([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4])
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

    up7 = Add()([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3])
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

    up8 = Add()([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2])
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

    up9 = Add()([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1])
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
  
    model.compile(optimizer=Adam(lr=1e-5), loss='binary_crossentropy', metrics=[dice_coef,'accuracy'])

    return model

In [0]:
model = get_unet()

In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 1 0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, None, None, 3 320         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_21 (Conv2D)              (None, None, None, 3 9248        conv2d_20[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, None, None, 3 0           conv2d_21[0][0]                  
__________________________________________________________________________________________________
conv2d_22 

In [0]:
model_checkpoint1 = ModelCheckpoint('./models/U_net_1E_16B.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [0]:
classifier = model.fit(trainX,trainY, batch_size = 32, epochs = 50, verbose = True, validation_data = (testX, testY),callbacks=[TensorBoard(log_dir='/tmp/gated_cnn_autoencoder', histogram_freq=0,write_graph=True),model_checkpoint1])

Train on 1600 samples, validate on 400 samples
Epoch 1/50
1568/1600 [============================>.] - ETA: 18s - loss: 0.6870 - dice_coef: 0.2766 - acc: 0.8007
Epoch 00001: loss improved from inf to 0.68682, saving model to ./models/U_net_1E_16B.hdf5
1600/1600 [==============================] - 1004s 628ms/step - loss: 0.6868 - dice_coef: 0.2762 - acc: 0.8012 - val_loss: 0.6802 - val_dice_coef: 0.2846 - val_acc: 0.7952
Epoch 2/50
1568/1600 [============================>.] - ETA: 18s - loss: 0.6670 - dice_coef: 0.2750 - acc: 0.8068
Epoch 00002: loss improved from 0.68682 to 0.66652, saving model to ./models/U_net_1E_16B.hdf5
1600/1600 [==============================] - 997s 623ms/step - loss: 0.6665 - dice_coef: 0.2745 - acc: 0.8073 - val_loss: 0.6462 - val_dice_coef: 0.2815 - val_acc: 0.7984
Epoch 3/50
1568/1600 [============================>.] - ETA: 18s - loss: 0.5596 - dice_coef: 0.2348 - acc: 0.8075
Epoch 00003: loss improved from 0.66652 to 0.55808, saving model to ./models/U_net

Epoch 23/50
1568/1600 [============================>.] - ETA: 18s - loss: 0.2218 - dice_coef: 0.6831 - acc: 0.9230
Epoch 00023: loss did not improve
1600/1600 [==============================] - 1008s 630ms/step - loss: 0.2210 - dice_coef: 0.6823 - acc: 0.9233 - val_loss: 0.2222 - val_dice_coef: 0.6793 - val_acc: 0.9234
Epoch 24/50
1568/1600 [============================>.] - ETA: 18s - loss: 0.2064 - dice_coef: 0.7016 - acc: 0.9288
Epoch 00024: loss improved from 0.21798 to 0.20585, saving model to ./models/U_net_1E_16B.hdf5
1600/1600 [==============================] - 998s 623ms/step - loss: 0.2059 - dice_coef: 0.7022 - acc: 0.9290 - val_loss: 0.2136 - val_dice_coef: 0.7051 - val_acc: 0.9253
Epoch 25/50
1568/1600 [============================>.] - ETA: 18s - loss: 0.2011 - dice_coef: 0.7109 - acc: 0.9301
Epoch 00025: loss improved from 0.20585 to 0.20142, saving model to ./models/U_net_1E_16B.hdf5
1600/1600 [==============================] - 996s 623ms/step - loss: 0.2014 - dice_coef:

Epoch 47/50
1568/1600 [============================>.] - ETA: 18s - loss: 0.1709 - dice_coef: 0.7448 - acc: 0.9376
Epoch 00047: loss did not improve
1600/1600 [==============================] - 997s 623ms/step - loss: 0.1715 - dice_coef: 0.7446 - acc: 0.9373 - val_loss: 0.1841 - val_dice_coef: 0.7363 - val_acc: 0.9335
Epoch 48/50
1568/1600 [============================>.] - ETA: 18s - loss: 0.1718 - dice_coef: 0.7475 - acc: 0.9373
Epoch 00048: loss did not improve
1600/1600 [==============================] - 997s 623ms/step - loss: 0.1728 - dice_coef: 0.7474 - acc: 0.9368 - val_loss: 0.1833 - val_dice_coef: 0.7309 - val_acc: 0.9325
Epoch 49/50
1568/1600 [============================>.] - ETA: 18s - loss: 0.1750 - dice_coef: 0.7433 - acc: 0.9362
Epoch 00049: loss did not improve
1600/1600 [==============================] - 1018s 636ms/step - loss: 0.1749 - dice_coef: 0.7425 - acc: 0.9364 - val_loss: 0.1864 - val_dice_coef: 0.7347 - val_acc: 0.9310
Epoch 50/50
1568/1600 [================

In [0]:
pred = model.predict(testX,verbose="True")

In [0]:
print("[INFO] accuracy: {:.2f}%".format(accuracy*100))

[INFO] accuracy: 93.51%
